сначала нам надо понять, как лучше выводить матрицу в ascii символах. самые подходящие для визуализации - █ и ░. найдем их коды для chr

In [ ]:
ord('░')

In [ ]:
ord('█')

подгружаем и импортим все что надо (в edit -> notebook settings надо поставить gpu)

In [ ]:
!curl https://colab.chainer.org/install | sh -

In [ ]:
import cupy as cp

In [ ]:
import codecs

In [ ]:
import json

эти показатели можно изменять или брать вместо глайдера другие фигуры при достаточном размере поля. при выборе obj_hor и obj_vert ака горизонталь и вертикаль левого верхнего угла фигуры важно чтобы координаты этого угла были в матрице, а то ошибка будет))

In [ ]:
length = 10

In [ ]:
glider = [[0, 1], [1, 2], [2, 2], [2, 1], [2,0]]

In [ ]:
obj_hor = int(input())
obj_vert = int(input())

подготовительная работа по созданию матрицы с фигурой

In [ ]:
matrix = cp.zeros(length*length).reshape(length, length)

In [ ]:
for i in glider:
     matrix[obj_hor + i[0]][obj_vert + i[1]] = 1

# the game of life #

In [ ]:
#обозначаем границы каждой клетки
boundries = [[-1, -1], [-1, 0], [-1, 1], [1, 0], [0, -1],
             [0, 1], [1, 1], [1, -1]]
#обозначаем границы для верхней правой и нижней левой - они не попадают под первый 
#эксепшн
corners = [[0, 0], [0, 1], [0, length - 1], [length - 1, length - 1],
           [length - 2, length - 1], [length - 2, 0], [length - 2, 1],
           [length - 1, 1]]
numb = 0
row = 0
#матрица на случай первого эксепшна
exceptmatrix = cp.rot90(matrix, k=2, axes=(0, 1))
#новая матрица куда будет записываться новое состояние
newmatrix = cp.zeros(length * length).reshape(
    length, length)
for z in matrix:
    for j in z:
        try:
          # проверяем границы клетки. они хорошо проверяются только если клетка 
          # не стоит на правой или нижней границе - так срез берется плохо и индекс выходит 
          # out of range
            k = [1 for i in boundries if
                 matrix[row + i[0]][numb + i[1]] == 1.0]
        except IndexError:
            try:
              # обрабатываем первый out of range поворачивая матрицу на 90 градусов - 
              # так правая и нижняя граница станут левой и верхней и срезы возьмутся
                k = [1 for i in boundries if
                     exceptmatrix[
                         length - row - 1 + i[0]][
                         length - numb - 1 + i[1]] == 1.0]
            except IndexError:
              #остается две клетки которые не попадают под прошлые эксепшны - 
              #их обрабатываем считай вручную
                if row == length - 1 and numb == 0:
                    k = [1 for i in corners if matrix[i[0]][i[1]] == 1.0]
                else:
                    if row == 0 and numb == length - 1:
                        k = [1 for i in corners if matrix[i[1]][i[0]] == 1.0]
# если насчитали у клетки больше определенное число соседей, меняем их по правилам
        if len(k) == 3 or (
                len(k) == 2 and matrix[row, numb] == 1):
            newmatrix[row, numb] = 1.0
        numb += 1
    numb = 0
    row += 1
matrix = newmatrix

#вывод матрицы в ?ascii? символах
for i in matrix:
  for j in i:
    if j == 1.0:
      print (chr(9608), end = '')
    else:
      print(chr(9617), end = '')
  print('\n')
    

запомнить матрицу

In [ ]:
filename = 'record.json'
b = matrix.tolist()

with open(filename, 'w') as file:
  file_path = "/path.json"
  json.dump(b, codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

 # for i in matrix:
  
  #  for j in i:
  #    json.dump(int(j), file)

вспомнить матрицу

In [ ]:
obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
b_new = json.loads(obj_text)
matrix = cp.array(b_new)